In [1]:
import numpy as np
import librosa
import soundfile as sf
import os
import random
import concurrent.futures

In [2]:
def pitch_shift(path, save_path, eps=1e-3): # path ——> spk_id
    spk_list = os.listdir(path)
    for spk in spk_list:
        seg_list = os.listdir(os.path.join(path, spk))
        for seg in seg_list:
            voice_list = os.listdir(os.path.join(path, spk, seg))
            for voice in voice_list:      
                try:
                    wav, sr  = librosa.load(os.path.join(path, spk, seg, voice), sr=16000)        
                    save_dir = os.path.join(save_path, spk, seg)
                    if not os.path.isdir(save_dir):
                        os.makedirs(save_dir)  
                    
                    # donot shift:
                    sf.write(os.path.join(save_dir, voice), wav, 16000)
            
                    # raise:
                    s=np.random.uniform(low=eps, high=12.0, size=None)
                    wav_r = librosa.effects.pitch_shift(wav, sr=22050, n_steps=s, bins_per_octave=12)
                    sf.write(os.path.join(save_dir, voice.split('.')[0]+'_'+str(round(s,3))+'_'+'.wav'), wav_r, 16000)
            
                    # lower:
                    s=np.random.uniform(low=-12.0, high=-eps, size=None)
                    wav_l = librosa.effects.pitch_shift(wav, sr=22050, n_steps=s, bins_per_octave=12)
                    sf.write(os.path.join(save_dir, voice.split('.')[0]+'_'+str(round(s,3))+'_'+'.wav'), wav_l, 16000)
                except:
                    print(os.path.join(path, spk, seg, voice))

In [3]:
path = '/root/autodl-tmp/Vox1/train'
save_path = '/root/autodl-tmp/Vox1/train_scaled'

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    executor.map(pitch_shift(path, save_path), range(128))

In [ ]:
import multiprocessing


# 设置一个允许3个进程并发的进程池
pool = multiprocessing.Pool(processes = 60)
for i in range(5000):
	# 将进程仍入进程池，mission 后面的这个含有 i 的tuple 代表给mission的参数
	pool.apply_async(pitch_shift(path, save_path), (i))
	
# 扔了 1000个进程进进程池后，关闭进程池，不允许新的进程加入
pool.close()
# 运行进程池中的进程
pool.join()

In [ ]:
1